In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [42]:
movie_sample = pd.read_csv('/content/sample_submission.csv')
movie_sample.head()

,id,genre
0,57609,action
1,30111,action
2,36119,action
3,65901,action
4,41910,action


In [43]:
movie_sample['genre'].value_counts()

genre
action    36000
Name: count, dtype: int64

In [44]:
test_df = pd.read_csv('/content/test.csv')
print(test_df.shape)
test_df.head(10)

(36000, 4)


,id,movie_name,synopsis,genre
0,16863,A Death Sentence,"12 y.o. Ida's dad'll die without a DKK1,500,00...",action
1,48456,Intermedio,A group of four teenage friends become trapped...,action
2,41383,30 Chua Phai Tet,A guy left his home for 12 years till he came ...,action
3,84007,Paranoiac,A man long believed dead returns to the family...,action
4,40269,Ordinary Happiness,"After a deadly accident, Paolo comes back on E...",action
5,16524,The Syndicate,"An alcoholic, his wife, a down and out America...",action
6,21245,12 Depths,"""Struck by amnesia and framed for murder, a ma...",action
7,84681,The Feud,A mysterious man returns to his home in Iowa f...,action
8,73352,Cabin Fever: Patient Zero,When a group of friends enjoying a bachelor cr...,action
9,37966,Peregrina,"After a near death experience, a young woman b...",action


In [18]:
test_df['genre'].value_counts()

genre
action    36000
Name: count, dtype: int64

In [21]:
!pip install pandas

import pandas as pd

# Try to read the file, handling potential errors
try:
    train_df = pd.read_csv('/content/train.csv')
    print(train_df.shape)
    train_df.head()
except pd.errors.ParserError as e:
    print(f"An error occurred while reading the file: {e}")
    # Print the problematic line for inspection
    with open('/content/train.csv', 'r') as file:
        for i, line in enumerate(file):
            if i == 51154:  # Adjust the line number if needed
                print(line)
                break

(54000, 4)


In [22]:
train_df['genre'].value_counts()

genre
fantasy      5400
horror       5400
family       5400
scifi        5400
action       5400
crime        5400
adventure    5400
mystery      5400
romance      5400
thriller     5400
Name: count, dtype: int64

In [23]:
test_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36000 entries, 0 to 35999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          36000 non-null  int64 
 1   movie_name  36000 non-null  object
 2   synopsis    36000 non-null  object
 3   genre       36000 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.1+ MB


In [24]:
features = train_df['synopsis']
features.head()

0    A young scriptwriter starts bringing valuable ...
1    A director and her friends renting a haunted h...
2    This is an educational video for families and ...
3    Scientists working in the Austrian Alps discov...
4    Buy Day - Four Men Widely - Apart in Life - By...
Name: synopsis, dtype: object

In [25]:
target = train_df['genre']
target.value_counts()

genre
fantasy      5400
horror       5400
family       5400
scifi        5400
action       5400
crime        5400
adventure    5400
mystery      5400
romance      5400
thriller     5400
Name: count, dtype: int64

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(features, target, test_size=0.2, random_state=2309, stratify=target)
X_train

47386    A ragtag spaceship crew sent on a rescue missi...
53524    An experimental film about various situations ...
28242    A series of loosely-connected shorts comprised...
46397    A gang of teenage girls, looking for kicks and...
12784    An FBI-trained neuropsychologist teams up with...
                               ...                        
18945    A jungle expedition in South America battles g...
25834    Balachandran who had to sacrifice his childhoo...
8044     A new, single, young doctor comes to town, tak...
50091    A series of killings against homosexual people...
38171    A young writer takes to edit the autobiography...
Name: synopsis, Length: 43200, dtype: object

In [28]:
X_test = test_df['synopsis']
X_test.head()

0    12 y.o. Ida's dad'll die without a DKK1,500,00...
1    A group of four teenage friends become trapped...
2    A guy left his home for 12 years till he came ...
3    A man long believed dead returns to the family...
4    After a deadly accident, Paolo comes back on E...
Name: synopsis, dtype: object

In [29]:
y_test = test_df['genre']
y_test.value_counts()

genre
action    36000
Name: count, dtype: int64

In [30]:
X_train.shape, y_train.shape, X_validation.shape,  y_validation.shape, X_test.shape, y_test.shape

((43200,), (43200,), (10800,), (10800,), (36000,), (36000,))

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer(stop_words='english', max_df=100)

count_vector.fit(X_train)
X_train_count_vector = count_vector.transform(X_train)
X_validation_count_vector = count_vector.transform(X_validation)
X_test_count_vector = count_vector.transform(X_test)

X_train_count_vector.shape, X_validation_count_vector.shape, X_test_count_vector.shape

((43200, 39892), (10800, 39892), (36000, 39892))

In [32]:
X_test_count_vector.toarray()[:5]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [33]:
y_train

47386       horror
53524      fantasy
28242      fantasy
46397        crime
12784        scifi
           ...    
18945    adventure
25834      romance
8044       romance
50091        scifi
38171        crime
Name: genre, Length: 43200, dtype: object

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

lr = LogisticRegression(max_iter=5000)
lr.fit(X_train_count_vector, y_train)

# lr model estimation
validation_pred = lr.predict(X_validation_count_vector)
accuracy_score(y_validation, validation_pred)

0.2112962962962963

In [35]:
# lr model prediction accuracy

test_lr_pred = lr.predict(X_test_count_vector)
accuracy_score(y_test, test_lr_pred)

0.09513888888888888

In [36]:
# confusion_matrix

print(confusion_matrix(y_test, test_lr_pred))

[[3425 3412 3575 3562 3264 3834 3508 4483 3522 3415]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]]


In [37]:
# classification_report

print(classification_report(y_test, test_lr_pred))

              precision    recall  f1-score   support

      action       1.00      0.10      0.17     36000
   adventure       0.00      0.00      0.00         0
       crime       0.00      0.00      0.00         0
      family       0.00      0.00      0.00         0
     fantasy       0.00      0.00      0.00         0
      horror       0.00      0.00      0.00         0
     mystery       0.00      0.00      0.00         0
     romance       0.00      0.00      0.00         0
       scifi       0.00      0.00      0.00         0
    thriller       0.00      0.00      0.00         0

    accuracy                           0.10     36000
   macro avg       0.10      0.01      0.02     36000
weighted avg       1.00      0.10      0.17     36000



In [38]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()

dtc.fit(X_train_count_vector, y_train)

# dtc model estimation
validation_pred = dtc.predict(X_validation_count_vector)
accuracy_score(y_validation, validation_pred)

0.165

In [39]:
# dtr model prediction accuracy

test_dtc_pred = dtc.predict(X_test_count_vector)
accuracy_score(y_test, test_dtc_pred)

0.10202777777777777

In [40]:
# confusion_matrix

print(confusion_matrix(y_test, test_dtc_pred))

[[3673 3628 3676 3539 3123 3716 3176 3219 5532 2718]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0]]


In [41]:
# classification_report

print(classification_report(y_test, test_dtc_pred))

              precision    recall  f1-score   support

      action       1.00      0.10      0.19     36000
   adventure       0.00      0.00      0.00         0
       crime       0.00      0.00      0.00         0
      family       0.00      0.00      0.00         0
     fantasy       0.00      0.00      0.00         0
      horror       0.00      0.00      0.00         0
     mystery       0.00      0.00      0.00         0
     romance       0.00      0.00      0.00         0
       scifi       0.00      0.00      0.00         0
    thriller       0.00      0.00      0.00         0

    accuracy                           0.10     36000
   macro avg       0.10      0.01      0.02     36000
weighted avg       1.00      0.10      0.19     36000

